In [1]:
from dotenv import load_dotenv
import os

import pandas as pd

from langchain_community.chat_models import ChatLlamaCpp
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_together import ChatTogether
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.tools import tool
from langgraph.graph import START, StateGraph, END
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display
import re
import json
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.prebuilt import create_react_agent

from pydantic import BaseModel, Field
from typing import List, Dict
from enum import Enum


In [2]:
load_dotenv()

True

In [3]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [4]:
LOCAL_MODEL_PATH = "/Users/gautamnaik/models/Nous-Hermes-2-Mistral-7B-DPO.Q8_0.gguf"
# LOCAL_MODEL_PATH = "/Users/gautamnaik/models/llama-3-8B-function-calling-Q8_0.gguf"
# LOCAL_MODEL_PATH = "/Users/gautamnaik/models/functionary/functionary-7b-v2.q8_0.gguf"

In [7]:

tools = []

def get_local_llm():
    return ChatLlamaCpp(
        model_path=LOCAL_MODEL_PATH,
        temperature=0,
        max_tokens=512,
        n_ctx=2048,
        n_batch=512,
        n_threads=8,
        verbose=False,
    )

def get_google_llm():
    return ChatGoogleGenerativeAI(
        temperature=0,
        max_output_tokens=512,
        model="gemini-2.0-flash",
        # model="gemini-2.5-flash-preview-05-20",
        #  model= "gemini-2.5-pro-preview-05-06",
        verbose=False,
    )

def get_groq_llm():
    return ChatGroq(
        model="qwen-qwq-32b",
        temperature=0,
    )

def get_chat_together_llm():
    return ChatTogether(
        api_key=os.getenv("TOGETHER_AI_API_KEY"),
        temperature=0.0,
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"
    )

def get_openai_llm():
    return ChatOpenAI(
        model="gpt-4.1",
        temperature=0.0,
        api_key=os.getenv("OPENAI_API_KEY"),
        verbose=False,
    )


In [8]:
def pre_model_hook(args) -> None:
    """
    This function is called before the model is invoked.
    It can be used to set up any necessary state or context.
    """
    print("Pre-model hook called.", args)

class ClassificationResult(BaseModel):
    classification: str = Field(
        description="The classification result, one of: Anxiety, Depression, Both, Neither"
    )
    confidence: str = Field(
        description="The confidence level of the classification, one of: Low, Medium, High"
    )
    reasoning: str = Field(
        description="A concise explanation of why this classification was made"
    )
    supporting_symptoms: List[str] = Field(
        description="A list of brief symptoms supporting the classification"
    )


supervisor = create_react_agent(
    model=get_chat_together_llm(),
    tools=[],
    prompt=(
      """
        You are a clinical classification assistant trained to detect anxiety and depression from structured therapy session data. Your task is to review psychological symptoms, mental status, client quotes, and other clinical observations to determine whether the patient likely has:

        - Anxiety
        - Depression
        - Both anxiety and depression
        - Neither

        Base your reasoning on the GAD-7 and PHQ-9 criteria.

        Return your output as a structured JSON object with the following fields:
        {
        "classification": "<Anxiety | Depression | Both | Neither>",
        "confidence": "<Low | Medium | High>",
        "reasoning": "<A concise explanation of why this classification was made>",
        "supporting_symptoms": ["<brief symptom 1>", "<brief symptom 2>", ...]
        }
        Do not hallucinate diagnoses. Only rely on information present in the input.
    """
    ),
    name="supervisor",
    pre_model_hook=pre_model_hook,
    response_format=ClassificationResult.model_json_schema()
)

# for chunk in gad_7_scorer_agent.stream(
#     {"messages": [{"role": "user", "content": """ 
# ```json\n
#     '[{"therapy_session_number": 1, "client_id": "client1", "chief_complaint": "The client reported feeling anxious and stressed recently, which is unusual for him. He also mentioned a tendency to take on tasks and procrastinate, leading to feelings of being overwhelmed.", "symptoms": [{"description": "Anxiety and stress", "onset": "Approximately six weeks ago", "frequency": "Intermittent, with episodes lasting a couple of weeks", "ascendance": "The symptoms have leveled off after the initial episodes", "intensity": "Moderate", "duration": "Approximately six weeks", "quote": "\\"I probably, probably a couple of, maybe six weeks ago, something. Yeah, I see that as quite recent.\\""}], "diagnosis": [{"dsm_5_diagnosis": "Generalized Anxiety Disorder", "dsm_5_code": "300.02", "icd_10_code": "F41.1"}], "mood_and_affect": "The client\'s mood was anxious, and his affect was congruent with his reported experiences.", "speech_and_language": "The client\'s speech was coherent, structured, and pertinent to the context.", "thought_process_and_content": "The client\'s thought process was organized, and his content was relevant to the discussion.", "orientation": "The client was oriented to person, place, time, and situation.", "perceptual_disturbances": "No perceptual disturbances were reported or observed.", "cognition": "The client\'s cognitive functioning appeared intact, with no significant deficits in memory, attention, or concentration.", "insight": "The client demonstrated a good level of insight into his symptoms and behaviors, recognizing the impact of his anxiety and stress on his functioning.", "risks_or_safety_concerns": "No Indication of Risk", "hopelessness": "No hopelessness expressed or observed.", "suicidal_thoughts_or_attempts": "No suicidal thoughts or attempts reported or suspected.", "self_harm": "No self-harm reported or suspected.", "dangerous_to_others": "No danger to others reported or suspected.", "risk_quote": "NA", "response_to_treatment": "NA"}, {"therapy_session_number": 2, "client_id": "client1", "chief_complaint": "The client reported a significant reduction in anxiety and stress. He mentioned feeling more confident in managing tasks and setting boundaries.", "symptoms": [{"description": "Anxiety and stress", "onset": "Six weeks ago, significantly reduced in the past two weeks", "frequency": "Occasional, with noticeable improvement", "ascendance": "Symptoms have decreased following the application of therapy strategies", "intensity": "Mild", "duration": "Minimal symptoms over the past two weeks", "quote": "\\"I\\u2019ve noticed the anxiety and stress have lessened, especially after focusing on setting boundaries.\\""}], "diagnosis": [{"dsm_5_diagnosis": "Generalized Anxiety Disorder", "dsm_5_code": "300.02", "icd_10_code": "F41.1"}], "mood_and_affect": "The client\'s mood was positive, and his affect was congruent with his reported improvements.", "speech_and_language": "The client\'s speech was coherent, upbeat, and reflective of his progress.", "thought_process_and_content": "The client\'s thought process was organized, and his content was focused on the positive changes he has made.", "orientation": "The client was oriented to person, place, time, and situation.", "perceptual_disturbances": "No perceptual disturbances were reported or observed.", "cognition": "The client\'s cognitive functioning appeared intact, with no significant deficits in memory, attention, or concentration.", "insight": "The client demonstrated a strong level of insight into his progress, recognizing the positive impact of the strategies he has implemented.", "risks_or_safety_concerns": "No Indication of Risk", "hopelessness": "No hopelessness expressed or observed.", "suicidal_thoughts_or_attempts": "No suicidal thoughts or attempts reported or suspected.", "self_harm": "No self-harm reported or suspected.", "dangerous_to_others": "No danger to others reported or suspected.", "risk_quote": "NA", "response_to_treatment": "The client has shown significant improvement, actively participating in therapy and implementing the strategies discussed."}]'
# """}]}
# ):
#     pretty_print_messages(chunk)

In [9]:
res = supervisor.invoke(
    {"messages": [{"role": "user", "content": """
```json\n
'{\n    "chief_complaint": "The client reported a significant reduction in anxiety and stress. He mentioned feeling more confident in managing tasks and setting boundaries.",\n    "symptoms": {\n        "Symptom 1": {\n            "Description": "Anxiety and stress",\n            "Onset": "Six weeks ago, significantly reduced in the past two weeks",\n            "Frequency": "Occasional, with noticeable improvement",\n            "Ascendance": "Symptoms have decreased following the application of therapy strategies",\n            "Intensity": "Mild",\n            "Duration": "Minimal symptoms over the past two weeks",\n            "Quote (Symptom)": "\\"I\\u2019ve noticed the anxiety and stress have lessened, especially after focusing on setting boundaries.\\""\n        }\n    },\n    "mood_and_affect": "The client\'s mood was positive, and his affect was congruent with his reported improvements.",\n    "thought_content": "The client\'s thought process was organized, and his content was focused on the positive changes he has made.",\n    "cognition": "The client\'s cognitive functioning appeared intact, with no significant deficits in memory, attention, or concentration.",\n    "hopelessness": "No hopelessness expressed or observed.",\n    "suicidal_thoughts": "No suicidal thoughts or attempts reported or suspected.",\n    "sleep": "Improved sleep patterns were reported, with more restful nights.",\n    "diagnosis": {\n        "Diagnosis 1": {\n            "Description": "Generalized Anxiety Disorder",\n            "DSM- Code": "300.02",\n            "ICD- Code": "F41.1",\n            "Reasoning": "Although the client continues to show signs of anxiety, the symptoms have significantly reduced, indicating positive progress towards managing the disorder."\n        }\n    },\n    "client_quotes": {\n        "chief_complaint_quote": "\\"I\\u2019ve been feeling a lot better lately, more in control and less overwhelmed.\\"",\n        "symptom_quotes": [\n            "\\"I\\u2019ve noticed the anxiety and stress have lessened, especially after focusing on setting boundaries.\\""\n        ],\n        "risk_quote": "NA"\n    }\n}'
```
""" }]},
)

Pre-model hook called. {'messages': [HumanMessage(content='\n```json\n\n\'{\n    "chief_complaint": "The client reported a significant reduction in anxiety and stress. He mentioned feeling more confident in managing tasks and setting boundaries.",\n    "symptoms": {\n        "Symptom 1": {\n            "Description": "Anxiety and stress",\n            "Onset": "Six weeks ago, significantly reduced in the past two weeks",\n            "Frequency": "Occasional, with noticeable improvement",\n            "Ascendance": "Symptoms have decreased following the application of therapy strategies",\n            "Intensity": "Mild",\n            "Duration": "Minimal symptoms over the past two weeks",\n            "Quote (Symptom)": "\\"I\\u2019ve noticed the anxiety and stress have lessened, especially after focusing on setting boundaries.\\""\n        }\n    },\n    "mood_and_affect": "The client\'s mood was positive, and his affect was congruent with his reported improvements.",\n    "thought_c

In [ ]:
res

{'messages': [HumanMessage(content='\n```json\n\n\'{\n    "chief_complaint": "The client reported a significant reduction in anxiety and stress. He mentioned feeling more confident in managing tasks and setting boundaries.",\n    "symptoms": {\n        "Symptom 1": {\n            "Description": "Anxiety and stress",\n            "Onset": "Six weeks ago, significantly reduced in the past two weeks",\n            "Frequency": "Occasional, with noticeable improvement",\n            "Ascendance": "Symptoms have decreased following the application of therapy strategies",\n            "Intensity": "Mild",\n            "Duration": "Minimal symptoms over the past two weeks",\n            "Quote (Symptom)": "\\"I\\u2019ve noticed the anxiety and stress have lessened, especially after focusing on setting boundaries.\\""\n        }\n    },\n    "mood_and_affect": "The client\'s mood was positive, and his affect was congruent with his reported improvements.",\n    "thought_content": "The client\'s

In [18]:
res["messages"]

[HumanMessage(content='\n```json\n\n\'{\n    "chief_complaint": "The client reported a significant reduction in anxiety and stress. He mentioned feeling more confident in managing tasks and setting boundaries.",\n    "symptoms": {\n        "Symptom 1": {\n            "Description": "Anxiety and stress",\n            "Onset": "Six weeks ago, significantly reduced in the past two weeks",\n            "Frequency": "Occasional, with noticeable improvement",\n            "Ascendance": "Symptoms have decreased following the application of therapy strategies",\n            "Intensity": "Mild",\n            "Duration": "Minimal symptoms over the past two weeks",\n            "Quote (Symptom)": "\\"I\\u2019ve noticed the anxiety and stress have lessened, especially after focusing on setting boundaries.\\""\n        }\n    },\n    "mood_and_affect": "The client\'s mood was positive, and his affect was congruent with his reported improvements.",\n    "thought_content": "The client\'s thought proc

In [26]:
res["structured_response"]

{'classification': 'Anxiety',
 'confidence': 'High',
 'reasoning': "The client's symptoms and quotes indicate a significant reduction in anxiety, but the presence of mild anxiety symptoms and a diagnosis of Generalized Anxiety Disorder suggest that anxiety is still present.",
 'supporting_symptoms': ['Mild anxiety symptoms',
  'Reduced stress',
  'Improved sleep patterns',
  'Client quote: "I\'ve noticed the anxiety and stress have lessened, especially after focusing on setting boundaries."']}

In [13]:
type(res["structured_response"])

dict

In [14]:
ClassificationResult.model_validate(res["structured_response"])

ClassificationResult(classification='Anxiety', confidence='High', reasoning="The client's symptoms and quotes indicate a significant reduction in anxiety, but the presence of mild anxiety symptoms and a diagnosis of Generalized Anxiety Disorder (GAD) suggest that anxiety is still present. The client's positive mood, organized thought process, and lack of hopelessness or suicidal thoughts suggest that depression is not currently present.", supporting_symptoms=['Mild anxiety symptoms', 'Reduced stress', 'Improved sleep patterns', 'Increased confidence in managing tasks and setting boundaries'])

In [27]:
json.loads(res["structured_response"])

TypeError: the JSON object must be str, bytes or bytearray, not dict